In [11]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [12]:
# Function to load data from the database
def load_data():
    # Database connection parameters
    db_params = {
        'dbname': 'week1',
        'user': 'postgres',
        'password': 'habte',
        'host': 'localhost',
        'port': '5432'
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

    # SQL query to retrieve data
    sql_query = "SELECT * FROM xdr_data;"

    # Read data from PostgreSQL into a DataFrame
    mydata = pd.read_sql_query(sql_query, engine)

    return mydata

# Function to perform EDA and visualization
def perform_eda(data):
    # Display basic information about the dataset
    st.subheader("Dataset Information")
    st.write(data.info())

    # Display summary statistics of numerical columns
    st.subheader("Summary Statistics")
    st.write(data.describe())

    # Check for missing values
    st.subheader("Missing Values")
    st.write(data.isnull().sum())

    # Visualize correlations using a heatmap
    st.subheader("Correlation Matrix")
    
    # Exclude non-numeric columns from the correlation matrix
    numeric_data = data.select_dtypes(include=['float64', 'int64'])
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
    st.pyplot()

    # Visualize the distribution of a numerical column (e.g., 'Dur. (ms)')
    st.subheader("Distribution of Dur. (ms)")
    plt.figure(figsize=(10, 6))
    sns.histplot(data['Dur. (ms)'], bins=30, kde=True)
    st.pyplot()


# Function to perform Top Handsets Analysis
def top_handsets_analysis(data):
    # Display the top handsets based on usage
    st.subheader("Top Handsets Analysis")
    
    # Assuming 'Handset Type' is the column containing handset information
    top_handsets = data['Handset Type'].value_counts().head(10)
    st.bar_chart(top_handsets)

# Main Streamlit application
def main():
    st.title("Telecom User Overview Analysis")
    
    # Load data
    data = load_data()

    # Sidebar for user input (if needed)
    # ...

    # Navigation
    analysis_task = st.sidebar.radio("Select Analysis Task", ["Perform EDA", "Top Handsets Analysis"])

    if analysis_task == "Perform EDA":
        # Perform Exploratory Data Analysis
        perform_eda(data)
    elif analysis_task == "Top Handsets Analysis":
        # Perform Top Handsets Analysis
        top_handsets_analysis(data)
        
if __name__ == "__main__":
    main()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

/home/habte/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


<Figure size 1000x800 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [13]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Function to load data from the database
def load_data():
    # Database connection parameters
    db_params = {
        'dbname': 'week1',
        'user': 'postgres',
        'password': 'habte',
        'host': 'localhost',
        'port': '5432'
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

    # SQL query to retrieve data
    sql_query = "SELECT * FROM xdr_data;"

    # Read data from PostgreSQL into a DataFrame
    mydata = pd.read_sql_query(sql_query, engine)

    return mydata

# Function to identify the top N items in a column
def top_n_items(data, column, n=10):
    return data[column].value_counts().head(n)

# Function to perform Top Handsets Analysis
def top_handsets_analysis(data):
    # Display the top handsets based on usage
    st.subheader("Top 10 Handsets Used by Customers")
    
    top_handsets = top_n_items(data, 'Handset Type', 10)
    st.bar_chart(top_handsets)

# Function to identify the top N manufacturers
def top_n_manufacturers(data, column, n=3):
    return data[column].value_counts().head(n)

# Function to identify the top N handsets per manufacturer
def top_n_handsets_per_manufacturer(data, manufacturer_column, handset_column, n=5):
    top_manufacturers = top_n_manufacturers(data, manufacturer_column)
    
    for manufacturer in top_manufacturers.index:
        st.subheader(f"Top {n} Handsets for {manufacturer}")
        manufacturer_data = data[data[manufacturer_column] == manufacturer]
        top_handsets = top_n_items(manufacturer_data, handset_column, n)
        st.bar_chart(top_handsets)

# Main Streamlit application
def main():
    st.title("Telecom User Overview Analysis")
    
    # Load data
    data = load_data()

    # Sidebar for user input (if needed)
    # ...

    # Navigation
    analysis_task = st.sidebar.radio("Select Analysis Task", ["Top Handsets Analysis", "Top Manufacturers Analysis"])

    if analysis_task == "Top Handsets Analysis":
        # Perform Top Handsets Analysis
        top_handsets_analysis(data)
    elif analysis_task == "Top Manufacturers Analysis":
        # Perform Top Manufacturers Analysis
        st.subheader("Top 3 Handset Manufacturers")
        top_manufacturers = top_n_manufacturers(data, 'Manufacturer', 3)
        st.bar_chart(top_manufacturers)
        
        # Identify the top 5 handsets per top 3 manufacturers
        top_n_handsets_per_manufacturer(data, 'Manufacturer', 'Handset Type', 5)
        
if __name__ == "__main__":
    main()


In [15]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Function to load data from the database
def load_data():
    # Database connection parameters
    db_params = {
        'dbname': 'week1',
        'user': 'postgres',
        'password': 'habte',
        'host': 'localhost',
        'port': '5432'
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

    # SQL query to retrieve data
    sql_query = "SELECT * FROM xdr_data;"

    # Read data from PostgreSQL into a DataFrame
    mydata = pd.read_sql_query(sql_query, engine)

    return mydata

# Function to perform EDA and visualization
def perform_eda(data):
    # Display basic information about the dataset
    st.subheader("Dataset Information")
    st.write(data.info())

    # Display summary statistics of numerical columns
    st.subheader("Summary Statistics")
    st.write(data.describe())

    # Check for missing values
    st.subheader("Missing Values")
    st.write(data.isnull().sum())

    # Visualize correlations using a heatmap
    st.subheader("Correlation Matrix")

    # Exclude non-numeric columns from the correlation matrix
    numeric_data = data.select_dtypes(include=['float64', 'int64'])

    plt.figure(figsize=(10, 8))
    sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
    st.pyplot()

    # Visualize the distribution of a numerical column (e.g., 'Dur. (ms)')
    st.subheader("Distribution of Dur. (ms)")
    plt.figure(figsize=(10, 6))
    sns.histplot(data['Dur. (ms)'], bins=30, kde=True)
    st.pyplot()

# Function to perform Top Handsets Analysis
def top_handsets_analysis(data):
    # Display the top handsets based on usage
    st.subheader("Top Handsets Analysis")
    
    # Assuming 'Handset Type' is the column containing handset information
    top_handsets = data['Handset Type'].value_counts().head(10)
    st.bar_chart(top_handsets)

# Main Streamlit application
def main():
    st.title("Telecom User Overview Analysis")
    
    # Load data
    mydata = load_data()

    # Sidebar for user input (if needed)
    # ...

    # Navigation
    analysis_task = st.sidebar.radio("Select Analysis Task", ["Perform EDA", "Top Handsets Analysis"])

    if analysis_task == "Perform EDA":
        # Perform Exploratory Data Analysis
        perform_eda(mydata)
    elif analysis_task == "Top Handsets Analysis":
        # Perform Top Handsets Analysis
        top_handsets_analysis(mydata)
        
if __name__ == "__main__":
    main()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

/home/habte/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
